In [1]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from torchvision.datasets import ImageFolder
import flwr as fl
from torchsummary import summary
DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)
from sklearn.metrics import classification_report
from flwr.common import NDArrays, Scalar

from typing import Dict, Optional, Tuple

Training on cuda using PyTorch 2.0.0+cu118 and Flower 1.4.0


In [2]:
os.listdir()

['.ipynb_checkpoints', 'federated-bs80k.ipynb']

In [3]:
NUM_CLIENTS = 4
hw = 128

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize(size = (hw,hw), antialias=True),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5),
    ]
)

In [5]:
def findConv2dOutShape(hin,win,conv,pool=2):
    # get conv arguments
    kernel_size=conv.kernel_size
    stride=conv.stride
    padding=conv.padding
    dilation=conv.dilation

    hout=np.floor((hin+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
    wout=np.floor((win+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)

    if pool:
        hout/=pool
        wout/=pool
    return int(hout),int(wout)

class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        Cin,Hin,Win= (3, hw, hw)
        init_f= 32
        num_fc1= 512
        num_classes= 2
        self.dropout_rate= 0.2
        
        # Convolution Layers
        self.conv1 = nn.Conv2d(Cin, init_f, kernel_size=3)
        h,w=findConv2dOutShape(Hin,Win,self.conv1)
        self.conv2 = nn.Conv2d(init_f, 2*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv2)
        self.conv3 = nn.Conv2d(2*init_f, 4*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv3)
        self.conv4 = nn.Conv2d(4*init_f, 8*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv4)
        self.conv5 = nn.Conv2d(8*init_f, 16*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv5)
        
        self.num_flatten=h*w*16*init_f
        self.fc1 = nn.Linear(self.num_flatten, num_fc1)
        self.fc2 = nn.Linear(num_fc1, num_classes)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        # Convolution & Pool Layers
        X = F.relu(self.conv1(X)); 
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv4(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv5(X))
        X = F.max_pool2d(X, 2, 2) 
        
        X = X.view(-1, self.num_flatten)
        
        X = F.relu(self.fc1(X))
        X=F.dropout(X, self.dropout_rate)
        X = self.fc2(X)
        return F.log_softmax(X, dim=1)


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int, cid=None):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    
    with open("client-log-"+str(cid)+".txt", "a") as file:
        file.write("=======ROUND START======= \n")

    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        
        with open("client-log-"+str(cid)+".txt", "a") as file:
            file.write(f"{epoch_loss},{epoch_acc} \n")
            
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")

            


def test(net, testloader, cid=None):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    y_test = []
    y_pred = []
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            y_test = y_test + labels.cpu().numpy().tolist()
            y_pred = y_pred + predicted.cpu().numpy().tolist()
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    
    if cid == 'SERVER':
        with open("server-report-.txt", "a") as file:
            file.write(classification_report(y_test, y_pred, zero_division=0) + "\n")
    else:
        with open("client-report-"+str(cid)+".txt", "a") as file:
            file.write(classification_report(y_test, y_pred, zero_division=0) + "\n")
    
    
    accuracy = correct / total
    return loss, accuracy

In [6]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=10, cid=self.cid)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader, cid=self.cid)
        with open("client-result-"+str(self.cid)+".txt", "a") as file:
            file.write(f"{loss}, {accuracy} \n")
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)

    train_dataset = ImageFolder(root='D:/feda/partitions/client-'+str(cid)+'/split/train',
        transform=transform)
    val_dataset = ImageFolder(root='D:/feda/partitions/client-'+str(cid)+'/split/val',
        transform=transform)
    
    
    trainDataLoader = DataLoader(train_dataset, 
            batch_size=32, shuffle=True)
    valDataLoader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    return FlowerClient(cid, net, trainDataLoader, valDataLoader)

In [7]:
def get_evaluate_fn(model):
    
    val_dataset = ImageFolder(root='D:/feda/partitions/server',
        transform=transform)
    
    
    valDataLoader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    """Return an evaluation function for server-side evaluation."""

    model = Net().to(DEVICE)
    # The `evaluate` function will be called after every round
    def evaluate(
        server_round: int, parameters: NDArrays, config: Dict[str, Scalar]
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        set_parameters(model, parameters)  # Update model with the latest parameters
        loss, accuracy = test(model, valDataLoader, cid="SERVER")
        print("SERVER EVAL", accuracy, loss)
        with open("server-eval.txt", "a") as file:
            file.write(f"{loss}, {accuracy} \n")
        
        return loss, {"accuracy": accuracy}

    return evaluate

In [8]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.5,
    fraction_evaluate=0.5,
    min_fit_clients=2,
    min_evaluate_clients=2,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
    evaluate_fn=get_evaluate_fn(Net()),
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=50),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-05-09 19:24:57,356 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=50, round_timeout=None)
2023-05-09 19:24:59,764	INFO worker.py:1625 -- Started a local Ray instance.
INFO flwr 2023-05-09 19:25:02,632 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 2906456064.0, 'GPU': 1.0, 'CPU': 8.0, 'memory': 5812912128.0}
INFO flwr 2023-05-09 19:25:02,632 | server.py:86 | Initializing global parameters
INFO flwr 2023-05-09 19:25:02,633 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-05-09 19:25:02,634 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-05-09 19:25:05,113 | server.py:91 | initial parameters (loss, other metrics): 0.021915677851170635, {'accuracy': 0.5}
INFO flwr 2023-05-09 19:25:05,114 | server.py:101 | FL starting
DEBUG flwr 2023-05-09 19:25:05,114 | server.py:218 | fit_round 1: strategy sampled 2 clients (out of 4)


SERVER EVAL 0.5 0.021915677851170635
(launch_and_fit pid=18108) [Client 2] fit, config: {}
(launch_and_fit pid=18108) Epoch 1: train loss 0.02203908935189247, accuracy 0.4991212653778559
(launch_and_fit pid=18108) Epoch 2: train loss 0.021719178184866905, accuracy 0.5641476274165202
(launch_and_fit pid=18108) Epoch 3: train loss 0.02212834171950817, accuracy 0.5518453427065027
(launch_and_fit pid=18108) Epoch 4: train loss 0.021225983276963234, accuracy 0.5940246045694201
(launch_and_fit pid=18108) Epoch 5: train loss 0.02119523473083973, accuracy 0.5817223198594025
(launch_and_fit pid=18108) Epoch 6: train loss 0.019577888771891594, accuracy 0.6695957820738138
(launch_and_fit pid=18108) Epoch 7: train loss 0.01794556900858879, accuracy 0.7390158172231986
(launch_and_fit pid=18108) Epoch 8: train loss 0.019028663635253906, accuracy 0.7144112478031635
(launch_and_fit pid=18108) Epoch 9: train loss 0.01735183596611023, accuracy 0.7381370826010545
(launch_and_fit pid=18108) Epoch 10: trai

DEBUG flwr 2023-05-09 19:26:00,070 | server.py:232 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-05-09 19:26:00,100 | fedavg.py:243 | No fit_metrics_aggregation_fn provided


(launch_and_fit pid=20708) Epoch 10: train loss 0.018586529418826103, accuracy 0.6933216168717048


INFO flwr 2023-05-09 19:26:02,076 | server.py:119 | fit progress: (1, 0.022514584233586706, {'accuracy': 0.5070224719101124}, 56.9613662)
DEBUG flwr 2023-05-09 19:26:02,076 | server.py:168 | evaluate_round 1: strategy sampled 2 clients (out of 4)


...

(launch_and_fit pid=1808) Epoch 10: train loss 0.0021387215238064528, accuracy 0.9753954305799648


INFO flwr 2023-05-09 20:20:10,500 | server.py:119 | fit progress: (50, 0.030040450383689295, {'accuracy': 0.7710674157303371}, 3305.3857603)
DEBUG flwr 2023-05-09 20:20:10,501 | server.py:168 | evaluate_round 50: strategy sampled 2 clients (out of 4)


SERVER EVAL 0.7710674157303371 0.030040450383689295
(launch_and_evaluate pid=20524) [Client 2] evaluate, config: {}


DEBUG flwr 2023-05-09 20:20:19,475 | server.py:182 | evaluate_round 50 received 2 results and 0 failures
INFO flwr 2023-05-09 20:20:19,476 | server.py:147 | FL finished in 3314.3615896
INFO flwr 2023-05-09 20:20:19,478 | app.py:218 | app_fit: losses_distributed [(1, 0.022378042236074702), (2, 0.016534719850633525), (3, 0.015597010930101358), (4, 0.01659630986673015), (5, 0.015596622275514203), (6, 0.013974840355502973), (7, 0.016863244917842895), (8, 0.015538677963968758), (9, 0.015213360319604408), (10, 0.015855104017716188), (11, 0.018993421138583363), (12, 0.016652207456893855), (13, 0.015519052274994085), (14, 0.018160648383460678), (15, 0.017526948941754297), (16, 0.023539153819942808), (17, 0.02184449795346994), (18, 0.021858144434286162), (19, 0.020721795441179008), (20, 0.024597323779668008), (21, 0.029558954486896938), (22, 0.02089714757785514), (23, 0.026374897170629533), (24, 0.022295996025725677), (25, 0.023009002703678356), (26, 0.021553514564370775), (27, 0.02570410315077

History (loss, distributed):
	round 1: 0.022378042236074702
	round 2: 0.016534719850633525
	round 3: 0.015597010930101358
	round 4: 0.01659630986673015
	round 5: 0.015596622275514203
	round 6: 0.013974840355502973
	round 7: 0.016863244917842895
	round 8: 0.015538677963968758
	round 9: 0.015213360319604408
	round 10: 0.015855104017716188
	round 11: 0.018993421138583363
	round 12: 0.016652207456893855
	round 13: 0.015519052274994085
	round 14: 0.018160648383460678
	round 15: 0.017526948941754297
	round 16: 0.023539153819942808
	round 17: 0.02184449795346994
	round 18: 0.021858144434286162
	round 19: 0.020721795441179008
	round 20: 0.024597323779668008
	round 21: 0.029558954486896938
	round 22: 0.02089714757785514
	round 23: 0.026374897170629533
	round 24: 0.022295996025725677
	round 25: 0.023009002703678356
	round 26: 0.021553514564370775
	round 27: 0.025704103150776217
	round 28: 0.022954611863185474
	round 29: 0.024666515446001
	round 30: 0.029424614631212678
	round 31: 0.0372209560152